# 🎧 Moodle to MP3 Converter (Colab Edition)

**Steps:**
1. Get your Moodle cookies (one-time setup)
2. Extract video links from the course media page
3. Paste cookies and links below
4. Run all cells
5. Download your MP3 files!

---

### 🍪 Step A: Get Cookies (One-time setup)

1. Install **[Get cookies.txt LOCALLY](https://chromewebstore.google.com/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)** extension
2. Go to your Moodle site and **log in**
3. Click the extension icon → Click **"Export"**
4. The cookies are now in your clipboard!

---

### 🔗 Step B: Extract Video Links

1. Go to your course's **Media page** (where you see the recordings table)
2. Press **F12** to open DevTools → **Console** tab
3. If prompted, type `allow pasting` and press Enter
4. Paste this script and press Enter:

```javascript
const tableBody = document.querySelector("#videoslist_table > tbody");
const links = Array.from(tableBody.querySelectorAll("a"));
const videoUrls = links
    .map(a => a.href)
    .filter(href => href.includes("php") || href.includes("video"));
console.log(videoUrls.join('\n'));
copy(videoUrls.join('\n'));
console.log("✅ Links copied to clipboard!");
```

5. Links are copied - paste them in Step 2 below!
6. **For multiple pages:** Repeat steps 4-5 on each page, then combine all links

In [6]:
# Step 0: Install yt-dlp
!pip install -U yt-dlp -q
print("✅ yt-dlp ready!")

In [7]:
# Step 1: Paste your cookies here (from the extension)
# Click the extension icon on your Moodle page → Export → Paste below

cookies = """
# Netscape HTTP Cookie File
# Paste your cookies here - replace this example!
.example.ac.il	TRUE	/	FALSE	1769972716	_gid	GA1.3.1234567890.1769872127
moodle.example.ac.il	FALSE	/	FALSE	0	MoodleSession	YOUR_SESSION_HERE
"""

# Save cookies to file
with open('cookies.txt', 'w') as f:
    f.write(cookies.strip())

# Verify cookies look valid
lines = [l for l in cookies.strip().split('\n') if l and not l.startswith('#')]
if lines:
    print(f"✅ Saved {len(lines)} cookie entries to cookies.txt")
else:
    print("⚠️ No cookies found! Make sure you pasted the cookies above.")

In [8]:
# Step 2: Paste your video links here (one per line)
# Use the console script from Step B above

links = """
https://moodle.example.ac.il/moodle/blocks/video/viewvideo.php?id=123456&courseid=12345&type=2
https://moodle.example.ac.il/moodle/blocks/video/viewvideo.php?id=123457&courseid=12345&type=2
"""

# Save to file
clean_links = [l.strip() for l in links.strip().split('\n') if l.strip() and not l.startswith('#')]
with open('links.txt', 'w') as f:
    f.write('\n'.join(clean_links))
print(f"✅ Saved {len(clean_links)} video links")

In [9]:
# Step 3: Download and convert to MP3
import os

if not os.path.exists('cookies.txt'):
    print("❌ cookies.txt not found! Run Step 1 first.")
elif not os.path.exists('links.txt'):
    print("❌ links.txt not found! Run Step 2 first.")
else:
    print("🍪 Using cookies for authentication...")
    print("📥 Downloading and converting to MP3...\n")
    !yt-dlp --cookies cookies.txt \
        -x --audio-format mp3 --audio-quality 64K \
        -o "%(autonumber)03d-%(title)s.%(ext)s" \
        --no-overwrites \
        --retries 5 \
        --concurrent-fragments 8 \
        -a links.txt
    print("\n✅ Download complete!")

In [10]:
# Step 4: Download your MP3 files
import glob
from google.colab import files
import zipfile

mp3_files = glob.glob('*.mp3')
print(f"📁 Found {len(mp3_files)} MP3 files")

if mp3_files:
    # Show file list
    for f in mp3_files:
        print(f"   • {f}")

    # Create zip
    with zipfile.ZipFile('moodle_audio.zip', 'w') as zipf:
        for mp3 in mp3_files:
            zipf.write(mp3)

    print("\n📦 Downloading moodle_audio.zip...")
    files.download('moodle_audio.zip')
else:
    print("❌ No MP3 files found. Check Step 3 for errors.")